In [ ]:
import pandas as pd
from tqdm.autonotebook import tqdm
from vertexai.preview.language_models import TextGenerationModel

In [ ]:
# Input parameter start here

In [ ]:
prompt_eval = """
<INSTRUCTIONS>
You are an expert analyst.
1. Given a question <QUESTION>, determine if <GENERATED ANSWER> is semantically equivalent to <GROUND TRUTH>
2. Given a question <QUESTION>, determine if <GENERATED ANSWER> contains all the key points in the <GROUND TRUTH>
3. First, think step-by-step and provide detailed reasoning <REASONING> for your analysis in Step 1 and Step 2. Explain which parts of the <GENERATED ANSWER> is different or similar to the <GROUND TRUTH>
4. Then reply with your final answer <FINAL ANSWER> which can only be [Yes, No, Unsure]
5. ALWAYS open and close all XML tags for example <FINAL ANSWER></FINAL ANSWER>
6. Reasoning must be written in English ONLY.
</INSTRUCTIONS>

<QUESTION>
{0}
</QUESTION>
<GROUND TRUTH>
{1}
</GROUND TRUTH>
<GENERATED ANSWER>
{2}
</GENERATED ANSWER>

Begin!
<REASONING>
"""

use_model = 'base'
model_name = 'text-bison@latest'

FILE_INPUT = ''

In [ ]:

FILE_OUTPUT = FILE_INPUT.split('.csv')[0]+'-output.csv'

In [ ]:
judge = TextGenerationModel.from_pretrained(model_name)

In [ ]:
df = pd.read_csv(FILE_INPUT)

In [ ]:
result_df = df.copy(deep=True)
result_df['judge_response'] = None
result_df['verdict'] = None

In [ ]:
for index, row in tqdm(df.iterrows()):

    input_prompt = prompt_eval.format(row['original_question'], row['ground_truth_answer'], row['generated_answer'])

    response = judge.predict(input_prompt, temperature=0, max_output_tokens=1024)

    # print(index, response.text)
    result_df.loc[index, 'judge_response'] = response.text
    result_df.loc[index, 'verdict'] = response.text.split('FINAL ANSWER')[1][1:-2]

In [ ]:
result_df.head()

In [ ]:
result_df.groupby('verdict').count()['judge_response'].plot(kind='pie', autopct='%1.0f%%')

In [ ]:
result_df.to_csv(FILE_OUTPUT, index=False)